# Import Libraries and Start Spark
____________

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.functions import udf, col, desc, asc
import pyspark.sql.functions as F
from pyspark.sql.functions import year, month, dayofmonth, \
    hour, weekofyear, date_format
from pyspark.sql.functions import isnan, col, when, count
from pyspark.context import SparkContext
from pyspark.sql.types import *
from pyspark.sql.types import StructType as R, \
            StructField as Fld, \
            DoubleType as Dbl,  \
            LongType as Long,   \
            StringType as Str,  \
            IntegerType as Int, \
            DecimalType as Dec, \
            DateType as Date,   \
            FloatType as Float, \
            TimestampType as Stamp
from pyspark.sql.window import Window as W

import datetime
import numpy as np
import os

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1665855331808_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'boto3'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'boto3'



In [2]:
# https://knowledge.udacity.com/questions/73278
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Add missing packages

In [4]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# sc.install_pypi_package("pip==22.1.2")

In [5]:
sc.install_pypi_package("cython==0.29.30")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# sc.install_pypi_package("pandas==1.3.5")
sc.install_pypi_package("pandas==0.19.2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sc.install_pypi_package("PyArrow==0.8.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
sc.install_pypi_package("geopy==2.2.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
import pandas as pd
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_columns', 500)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read from S3

## 1. State data (json)
https://worldpopulationreview.com/states/state-abbreviations

- read json file to df
- drop unneeded column(s)
- rename and re-case the remaining headers
- create an 'index'

In [19]:
# read json file to pyspark df

spark.conf.set("spark.sql.legacy.json.allowEmptyString.enabled", True)
states_data = spark.read \
    .option("multiline","true")\
    .json('s3://capstonesources/SOURCE/states.json')

states_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Abbrev: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- State: string (nullable = true)

In [20]:
# drop unneeded column
states_data = states_data.drop(col('Abbrev'))

states_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Code: string (nullable = true)
 |-- State: string (nullable = true)

In [21]:
# rename and re-case remaining headers

states_data = states_data \
    .withColumnRenamed("State", "state_name") \
    .withColumnRenamed("Code", "state_code") \

states_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- state_code: string (nullable = true)
 |-- state_name: string (nullable = true)

In [22]:
# create an 'index'

states_data = states_data \
    .withColumn("state_id", \
                F.monotonically_increasing_id())

win = W.orderBy("state_id")

states_data = states_data \
    .withColumn("state_id", \
                F.row_number().over(win)-1)

states_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- state_code: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- state_id: integer (nullable = true)

In [23]:
states_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------+
|state_code|          state_name|state_id|
+----------+--------------------+--------+
|        AL|             Alabama|       0|
|        AK|              Alaska|       1|
|        AZ|             Arizona|       2|
|        AR|            Arkansas|       3|
|        CA|          California|       4|
|        CO|            Colorado|       5|
|        CT|         Connecticut|       6|
|        DE|            Delaware|       7|
|        DC|District of Columbia|       8|
|        FL|             Florida|       9|
|        GA|             Georgia|      10|
|        HI|              Hawaii|      11|
|        ID|               Idaho|      12|
|        IL|            Illinois|      13|
|        IN|             Indiana|      14|
|        IA|                Iowa|      15|
|        KS|              Kansas|      16|
|        KY|            Kentucky|      17|
|        LA|           Louisiana|      18|
|        ME|               Maine|      19|
+----------

Validate

In [24]:
states_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

51

In [25]:
# count of nulls for all columns
check_emptyNullsNaNs = \
    states_data.select([count(when(isnan(c), c))\
                .alias(c) for c in states_data\
                .columns])\
                .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------+
|state_code|state_name|state_id|
+----------+----------+--------+
|         0|         0|       0|
+----------+----------+--------+

In [26]:
# states_data.show(5)
states_data.toPandas().sample(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   state_code  state_name  state_id
44         UT        Utah        44
31         NM  New Mexico        31
43         TX       Texas        43
0          AL     Alabama         0
45         VT     Vermont        45

## 2. Baby Names by State data (TXT)
https://www.ssa.gov/oact/babynames/limits.html

read multiple text files into a single df

In [27]:
namesbystate = spark.read \
    .option("header",False) \
    .text('s3://capstonesources/SOURCE/*.TXT')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

create a new schema
- split data to make columns
- drop unneeded columns
- trim leading and/or trailing spaces
- sort in ascending order
- create a unique record id
- reorder columns

In [28]:
# use split to make headers from existing columns

split_col = \
    F.split(namesbystate['value'], ',')

namesbystate = namesbystate \
    .withColumn('birth_year', \
                split_col.getItem(2) \
               )

namesbystate = namesbystate \
    .withColumn('popularity', \
                split_col.getItem(4) \
               )

namesbystate = namesbystate \
 .withColumn('birth_state_code', \
             split_col.getItem(0) \
            )

namesbystate = namesbystate \
    .withColumn('gender', \
                split_col.getItem(1) \
               )

namesbystate = namesbystate \
    .withColumn('birth_name', \
                split_col.getItem(3) \
               ) \

#drop unneeded column
namesbystate = namesbystate.drop('value')

namesbystate.printSchema()                                      

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- birth_year: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- birth_state_code: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_name: string (nullable = true)

In [29]:
# trim leading and/or trailing spaces

namesbystate = namesbystate\
    .withColumn('birth_state_code', \
                trim(namesbystate['birth_state_code']))

namesbystate = namesbystate\
    .withColumn('gender', \
                trim(namesbystate['gender']))

namesbystate = namesbystate\
    .withColumn('birth_year', \
                trim(namesbystate['birth_year']))

namesbystate = namesbystate\
 .withColumn('birth_name', \
            trim(namesbystate['birth_name'])) \

namesbystate = namesbystate\
 .withColumn('popularity', \
            trim(namesbystate['popularity']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# make birth_name lowercase

namesbystate = namesbystate\
 .withColumn('birth_name', \
            lower(col('birth_name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# sort in ascending order

namesbystate = \
    namesbystate.sort( \
    namesbystate['birth_year'].asc(), \
    namesbystate['birth_name'].asc(),
    namesbystate['popularity'].asc(), \
    namesbystate['birth_state_code'].asc(), \
#     namesbystate['gender'].asc() \
                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# create an 'index'

namesbystate = namesbystate \
    .withColumn("babyName_id", \
                F.monotonically_increasing_id())

win = W.orderBy("babyName_id")

namesbystate = namesbystate \
    .withColumn("babyName_id", \
                F.row_number().over(win)-1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# reorder df

namesbystate = namesbystate\
        .select('babyName_id', \
                'birth_year', \
                'popularity', \
                'birth_state_code', \
                'gender', \
                'birth_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
namesbystate.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+----------+----------------+------+----------+
|babyName_id|birth_year|popularity|birth_state_code|gender|birth_name|
+-----------+----------+----------+----------------+------+----------+
|          0|      1910|        13|              GA|     M|     aaron|
|          1|      1910|        15|              NY|     M|     aaron|
|          2|      1910|        15|              PA|     M|     aaron|
|          3|      1910|         5|              TN|     M|     aaron|
|          4|      1910|         5|              VA|     M|     aaron|
+-----------+----------+----------+----------------+------+----------+
only showing top 5 rows

Validate

In [35]:
namesbystate.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6311504

In [36]:
# count of nulls for all columns

check_emptyNullsNaNs = \
    namesbystate.select( \
            [count(when(isnan(c), c)) \
            .alias(c) for c in namesbystate\
            .columns] \
             ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+----------+----------------+------+----------+
|babyName_id|birth_year|popularity|birth_state_code|gender|birth_name|
+-----------+----------+----------+----------------+------+----------+
|          0|         0|         0|               0|     0|         0|
+-----------+----------+----------+----------------+------+----------+

In [37]:
# namesbystate.show(5)
namesbystate.toPandas().sample(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         babyName_id birth_year popularity birth_state_code gender  \
2740021      2740021       1975         60               IN      F   
576405        576405       1927          7               IN      M   
325528        325528       1920         18               KY      M   
258614        258614       1919         13               IL      M   
2677855      2677855       1974         11               AR      M   

          birth_name  
2740021        traci  
576405         lindy  
325528       wendell  
258614   christopher  
2677855         ross

# 3. Atlantic Hurricane Database 2 (HURDAT2) (txt)
HURDAT2 description: https://www.aoml.noaa.gov/hrd/hurdat/Data_Storm.html \
HURDAT2 legend: https://www.aoml.noaa.gov/hrd/hurdat/hurdat2-format.pdf \
HURDAT2 dataset: https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2021-041922.txt

Applying multiple operations to the raw Atlantic Hurricane 
Database 2 (HURDAT2 storm) dataset proved quite challenging in Pyspark. 

HURDAT2 storm data is a multiline text file with ~55K records and requires numerous 
complex transformations to be useful for basic analysis. 

By Spark standards, this is a small dataset.

I discovered using Pyspark to transform 
the HURDAT2 storm data seems to take disproportionately longer to 
complete relative to our other datasets, because of the 
excess overhead given the size of the cluster 
(cores and executors).

I abandon trying to transform the
raw HURDAT2 storm data in Pyspark.

Instead, for the HURDAT2 storm dataset, the more challenging 
transformations were performed using Pandas and
written into a csv file (see: capstoneSTORMS_CSV.ipynb). 

The csv file of semi-transformed HURDAT2 storm data ('storms.csv') 
was imported into my Spark session along with my other data sources. 

A few final data transformations are done using Pyspark - 
to demonstrate some aptitude with Pyspark - before building 
the schema tables and writing to S3. 

In [38]:
# read storms data from csv

fromStormsCSV = spark.read \
    .option("header", True) \
    .option("ignoreTrailingWhiteSpace",True) \
    .option("ignoreLeadingWhiteSpace", True) \
    .csv('s3://capstonesources/SOURCE/*.csv')

fromStormsCSV.printSchema()
fromStormsCSV.show(13)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- storm_id: string (nullable = true)
 |-- storm_name: string (nullable = true)
 |-- associated_records: string (nullable = true)
 |-- storm_date: string (nullable = true)
 |-- storm_time: string (nullable = true)
 |-- rec_identifier: string (nullable = true)
 |-- storm_type: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- max_sustained_wind(kt): string (nullable = true)
 |-- minimum_pressure(mbar): string (nullable = true)
 |-- category: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- storm_state_code: string (nullable = true)

+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|    storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|      cat

In [39]:
# check for NaNs

check_NaNs = \
    fromStormsCSV.select( \
            [count(when(isnan(c), c)) \
            .alias(c) for c in fromStormsCSV\
            .columns] \
             ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|category|coordinates|storm_state_code|
+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+
|       0|         0|                 0|         0|         0|             0|         0|       0|        0|                     0|                     0|       0|          0|               0|
+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+

In [40]:
# check for nulls

check_Nulls = \
    fromStormsCSV.select( \
            [count(when(isnull(c), c)) \
            .alias(c) for c in fromStormsCSV\
            .columns] \
             ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|category|coordinates|storm_state_code|
+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+
|       0|         0|                 0|         0|         0|         25900|         0|       0|        0|                     0|                     0|       0|          0|           26363|
+--------+----------+------------------+----------+----------+--------------+----------+--------+---------+----------------------+----------------------+--------+-----------+----------------+

In [41]:
# trim leading and/or trailing spaces

fromStormsCSV = fromStormsCSV\
    .withColumn('storm_id', \
                trim(col('storm_id'))) \
    .withColumn('storm_name', \
                trim(col('storm_name')), \
               ) \
    .withColumn('associated_records', \
                trim(col('associated_records')) \
               ) \
    .withColumn('storm_time', \
                trim(col('storm_time'))) \
    .withColumn('rec_identifier', \
                trim(col('rec_identifier'))) \
    .withColumn('storm_type', \
                trim(col('storm_type'))) \
    .withColumn('latitude', \
                trim(col('latitude'))) \
    .withColumn('longitude', \
                trim(col('longitude'))) \
    .withColumn('max_sustained_wind(kt)', \
                trim(col('max_sustained_wind(kt)')) \
               ) \
    .withColumn('minimum_pressure(mbar)', \
                trim(col('minimum_pressure(mbar)'))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# change to lowercase

fromStormsCSV = fromStormsCSV\
    .withColumn('storm_name', \
        lower(col('storm_name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# recast to str fields to int

fromStormsCSV = fromStormsCSV \
    .withColumn("associated_records", \
        fromStormsCSV["associated_records"] \
    .cast(IntegerType()))

fromStormsCSV = fromStormsCSV\
    .withColumn("max_sustained_wind(kt)",\
        fromStormsCSV["max_sustained_wind(kt)"]\
    .cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
fromStormsCSV.printSchema()
fromStormsCSV.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- storm_id: string (nullable = true)
 |-- storm_name: string (nullable = true)
 |-- associated_records: integer (nullable = true)
 |-- storm_date: string (nullable = true)
 |-- storm_time: string (nullable = true)
 |-- rec_identifier: string (nullable = true)
 |-- storm_type: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- max_sustained_wind(kt): integer (nullable = true)
 |-- minimum_pressure(mbar): string (nullable = true)
 |-- category: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- storm_state_code: string (nullable = true)

+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|    storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|      c

In [45]:
# check cast whether worked

# fromStormsCSV.select('associated_records').collect()[0][0]
# fromStormsCSV.select("max_sustained_wind(kt)").collect()[0][0]

fromStormsCSV.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- storm_id: string (nullable = true)
 |-- storm_name: string (nullable = true)
 |-- associated_records: integer (nullable = true)
 |-- storm_date: string (nullable = true)
 |-- storm_time: string (nullable = true)
 |-- rec_identifier: string (nullable = true)
 |-- storm_type: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- max_sustained_wind(kt): integer (nullable = true)
 |-- minimum_pressure(mbar): string (nullable = true)
 |-- category: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- storm_state_code: string (nullable = true)

In [46]:
# split storm_date into new column for storm_year

fromStormsCSV.select('storm_date').show(10,False)
split_col = F.split(fromStormsCSV['storm_date'], '-') 

fromStormsCSV = fromStormsCSV \
    .withColumn('storm_year', \
                trim(split_col.getItem(0)))

fromStormsCSV.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|storm_date|
+----------+
|1950-08-12|
|1950-08-12|
|1950-08-12|
|1950-08-12|
|1950-08-13|
|1950-08-13|
|1950-08-13|
|1950-08-13|
|1950-08-14|
|1950-08-14|
+----------+
only showing top 10 rows

+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|    storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|      category|  coordinates|storm_state_code|storm_year|
+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+
|AL011950|      able|                51|1950-08-12|  00:00:00|          null|tropical storm|    17.1|    -55.5|                    35|             

In [47]:
# create basin and ATCF_cyclone_num_forYear from the storm_id using substring

fromStormsCSV = fromStormsCSV\
    .withColumn('ATCF_cyclone_num_forYear', col('storm_id').substr(3, 2))\
    .withColumn('basin',col('storm_id').substr(1, 2))

fromStormsCSV.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+------------------------+-----+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|    storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|      category|  coordinates|storm_state_code|storm_year|ATCF_cyclone_num_forYear|basin|
+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+------------------------+-----+
|AL011950|      able|                51|1950-08-12|  00:00:00|          null|tropical storm|    17.1|    -55.5|                    35|                -999.0|tropical storm|(17.1, -55.5)|            null|      1950|                      01|   AL|
|AL011950|      

In [48]:
# filter for named storms

namedStorms = fromStormsCSV.filter(col('storm_name')!= 'unnamed')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# change data type for max_sustained_wind(kt) to float

namedStorms = namedStorms \
    .withColumn('max_sustained_wind(kt)' \
                ,(namedStorms['max_sustained_wind(kt)']).cast(Float())) 

namedStorms.select("max_sustained_wind(kt)").collect()[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

35.0

In [50]:
namedStorms.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+------------------------+-----+
|storm_id|storm_name|associated_records|storm_date|storm_time|rec_identifier|    storm_type|latitude|longitude|max_sustained_wind(kt)|minimum_pressure(mbar)|      category|  coordinates|storm_state_code|storm_year|ATCF_cyclone_num_forYear|basin|
+--------+----------+------------------+----------+----------+--------------+--------------+--------+---------+----------------------+----------------------+--------------+-------------+----------------+----------+------------------------+-----+
|AL011950|      able|                51|1950-08-12|  00:00:00|          null|tropical storm|    17.1|    -55.5|                  35.0|                -999.0|tropical storm|(17.1, -55.5)|            null|      1950|                      01|   AL|
|AL011950|      

# Build tables

![Schema](md_images/schemaCapstone.png) 


![DataDictionary](md_images/capstoneDataDict.png)

## saffir_simpson_hurricane_wind_scale_ref
https://www.nhc.noaa.gov/aboutsshws.php#:~:text=The%20Saffir%2DSimpson%20Hurricane%20Wind,Scale%20estimates%20potential%20property%20damage.

In [51]:
"""
NURDAT2 data contains no reference to
category. In order to identify category, 
the Saffir_Simpson Wind Scale must be 
referenced. The max_sustained_wind(kt)
can be measured against the min-max of 
each category's range to set the 
category, where the storm_type is 
hurricane

"""

data = [{'category': 1, 
         'sustained_wind(kt)': '64-82', 
         'max_sustained_wind(kt)': 82, 
         'min_sustained_wind(kt)': 64,
         'sustained_wind(mph)': '74-95', 
         'brief_damage_description': \
         'Power outages that could last a few to several days.'},
        
       {'category': 2, 
        'sustained_wind(kt)': '83-95', 
        'max_sustained_wind(kt)': 95, 
        'min_sustained_wind(kt)': 83,
        'sustained_wind(mph)': '96-110', 
        'brief_damage_description': \
        'Near-total power loss is expected \
        with outages that could last from several days to weeks.'},
        
       {'category': 3, 
        'sustained_wind(kt)': '96-112', 
        'max_sustained_wind(kt)': 112, 
        'min_sustained_wind(kt)': 96,
        'sustained_wind(mph)': '111-129', 
        'brief_damage_description': \
        'Electricity and water will be \
        unavailable for several days to weeks after the storm passes.'},
        
       {'category': 4,
        'sustained_wind(kt)': '113-136', 
        'max_sustained_wind(kt)': 136, 
        'min_sustained_wind(kt)': 113,
        'sustained_wind(mph)': '130-156', 
        'brief_damage_description': \
        'Catastrophic damage will occur; most of \
        the area will be uninhabitable for weeks or months.'},
        
       {'category': 5,
        'sustained_wind(kt)': '137+', 
        'min_sustained_wind(kt)': 137, 
        'sustained_wind(mph)': '157+',
        'brief_damage_description': \
        'Catastrophic damage will occur; most of the \
        area will be uninhabitable for weeks or months.'}]


schema = StructType([
    StructField('category', Int()),
    StructField('min_sustained_wind(kt)', Int()),
    StructField('max_sustained_wind(kt)', Int()),
    StructField('sustained_wind(kt)', Str()),
    StructField('brief_damage_description', Str())
])

# create data frame
saffir_simpson_scale = spark.createDataFrame(data, schema)
print(saffir_simpson_scale.printSchema)
saffir_simpson_scale.na.fill(value=0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<bound method DataFrame.printSchema of DataFrame[category: int, min_sustained_wind(kt): int, max_sustained_wind(kt): int, sustained_wind(kt): string, brief_damage_description: string]>
+--------+----------------------+----------------------+------------------+------------------------+
|category|min_sustained_wind(kt)|max_sustained_wind(kt)|sustained_wind(kt)|brief_damage_description|
+--------+----------------------+----------------------+------------------+------------------------+
|       1|                    64|                    82|             64-82|    Power outages tha...|
|       2|                    83|                    95|             83-95|    Near-total power ...|
|       3|                    96|                   112|            96-112|    Electricity and w...|
|       4|                   113|                   136|           113-136|    Catastrophic dama...|
|       5|                   137|                     0|              137+|    Catastrophic dama...|
+------

In [52]:
# # check data types

# saffir_simpson_scale.select('min_sustained_wind(kt)').collect()[1][0]
# saffir_simpson_scale.select('max_sustained_wind(kt)').collect()[1][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# write to s3
saffir_simpson_scale.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/saffir_simpson_scale/saffir_simpson_scale.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## state_ref

In [54]:
# transformation were handled when the json was read

state_ref = states_data

state_ref.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------+
|state_code|          state_name|state_id|
+----------+--------------------+--------+
|        AL|             Alabama|       0|
|        AK|              Alaska|       1|
|        AZ|             Arizona|       2|
|        AR|            Arkansas|       3|
|        CA|          California|       4|
|        CO|            Colorado|       5|
|        CT|         Connecticut|       6|
|        DE|            Delaware|       7|
|        DC|District of Columbia|       8|
|        FL|             Florida|       9|
|        GA|             Georgia|      10|
|        HI|              Hawaii|      11|
|        ID|               Idaho|      12|
|        IL|            Illinois|      13|
|        IN|             Indiana|      14|
|        IA|                Iowa|      15|
|        KS|              Kansas|      16|
|        KY|            Kentucky|      17|
|        LA|           Louisiana|      18|
|        ME|               Maine|      19|
+----------

In [55]:
# write to s3
 
state_ref.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/state_ref/state_ref.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## babyNames_byState_fact

In [56]:
# transformation were handled when the TXT files were read 

babyNames_byState_fact = namesbystate

babyNames_byState_fact.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+----------+----------------+------+----------+
|babyName_id|birth_year|popularity|birth_state_code|gender|birth_name|
+-----------+----------+----------+----------------+------+----------+
|          0|      1910|        13|              GA|     M|     aaron|
|          1|      1910|        15|              NY|     M|     aaron|
|          2|      1910|        15|              PA|     M|     aaron|
|          3|      1910|         5|              TN|     M|     aaron|
|          4|      1910|         5|              VA|     M|     aaron|
|          5|      1910|         5|              WV|     M|     aaron|
|          6|      1910|         6|              AR|     M|     aaron|
|          7|      1910|         6|              MO|     M|     aaron|
|          8|      1910|         7|              AL|     M|     aaron|
|          9|      1910|         8|              IL|     M|     aaron|
|         10|      1910|         8|              TX|     M|     aaron|
|     

In [57]:
# write to s3

babyNames_byState_fact.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/babyNames_byState_fact/babyNames_byState_fact.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## storms_byName_dim

In [58]:
storms_byName_dim = namedStorms \
    .select(['storm_id', 
             'basin',
             'ATCF_cyclone_num_forYear',
             'storm_year',
             'storm_name',
             'associated_records'
            ]) \
    .dropDuplicates()

storms_byName_dim.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+------------------------+----------+----------+------------------+
|storm_id|basin|ATCF_cyclone_num_forYear|storm_year|storm_name|associated_records|
+--------+-----+------------------------+----------+----------+------------------+
|AL021951|   AL|                      02|      1951|      able|                36|
|AL101955|   AL|                      10|      1955|     janet|                40|
|AL201995|   AL|                      20|      1995| sebastien|                19|
|AL161969|   AL|                      16|      1969|     gerda|                23|
|AL051999|   AL|                      05|      1999|    dennis|                66|
|AL072016|   AL|                      07|      2016|    gaston|                53|
|AL032019|   AL|                      03|      2019|     three|                 5|
|AL061984|   AL|                      06|      1984|    arthur|                33|
|AL051997|   AL|                      05|      1997|     danny|                47|
|AL1

In [59]:
# write to s3

storms_byName_dim.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/storms_byName_dim/storms_byName_dim.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## storms_location_dim

In [60]:
# select named storm that have storm_state_codes

storms_location_dim = namedStorms \
    .select(['storm_id', 
            'storm_name',
            'storm_year',
            'storm_type',
            'storm_state_code' 
            ]) \
    .where(namedStorms['storm_state_code'].isNotNull()) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# groupby the max storm_year and drop dups
storms_location_dim = storms_location_dim \
    .groupBy(['storm_id',
             'storm_name',
             'storm_type',
             'storm_state_code'
             ]).agg(F.max("storm_year"))\
    .dropDuplicates() 

storms_location_dim.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-------------------+----------------+---------------+
|storm_id|storm_name|storm_type         |storm_state_code|max(storm_year)|
+--------+----------+-------------------+----------------+---------------+
|AL092017|harvey    |hurricane          |TX              |2017           |
|AL112017|irma      |hurricane          |FL              |2017           |
|AL041985|danny     |hurricane          |LA              |1985           |
|AL051961|esther    |tropical storm     |MA              |1961           |
|AL092011|irene     |tropical storm     |NY              |2011           |
|AL111995|jerry     |tropical storm     |FL              |1995           |
|AL051960|donna     |hurricane          |NC              |1960           |
|AL011995|allison   |tropical storm     |FL              |1995           |
|AL051958|ella      |tropical storm     |TX              |1958           |
|AL192020|sally     |tropical depression|FL              |2020           |
|AL071956|flossy    |hurr

In [62]:
# create an 'index'

storms_location_dim = storms_location_dim \
    .withColumn("location_id", \
                F.monotonically_increasing_id())

win = W.orderBy("location_id")

storms_location_dim = storms_location_dim \
    .withColumn("location_id", \
                F.row_number().over(win)-1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# reorder and show

storms_location_dim = storms_location_dim \
    .select([
        'location_id', 
        'storm_id',
        'storm_name',
        'max(storm_year)',
        'storm_type',
        'storm_state_code' \
    ])

storms_location_dim.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+----------+---------------+-------------------+----------------+
|location_id|storm_id|storm_name|max(storm_year)|         storm_type|storm_state_code|
+-----------+--------+----------+---------------+-------------------+----------------+
|          0|AL092017|    harvey|           2017|          hurricane|              TX|
|          1|AL112017|      irma|           2017|          hurricane|              FL|
|          2|AL041985|     danny|           1985|          hurricane|              LA|
|          3|AL051961|    esther|           1961|     tropical storm|              MA|
|          4|AL092011|     irene|           2011|     tropical storm|              NY|
|          5|AL111995|     jerry|           1995|     tropical storm|              FL|
|          6|AL051960|     donna|           1960|          hurricane|              NC|
|          7|AL011995|   allison|           1995|     tropical storm|              FL|
|          8|AL051958|      ella|          

In [64]:
# reorder and show

storms_location_dim = storms_location_dim \
    .select([
        'location_id', 
        'storm_id',
        'storm_name',
        'max(storm_year)',
        'storm_type',
        'storm_state_code' \
    ])

storms_location_dim.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+----------+---------------+-------------------+----------------+
|location_id|storm_id|storm_name|max(storm_year)|         storm_type|storm_state_code|
+-----------+--------+----------+---------------+-------------------+----------------+
|          0|AL092017|    harvey|           2017|          hurricane|              TX|
|          1|AL112017|      irma|           2017|          hurricane|              FL|
|          2|AL041985|     danny|           1985|          hurricane|              LA|
|          3|AL051961|    esther|           1961|     tropical storm|              MA|
|          4|AL092011|     irene|           2011|     tropical storm|              NY|
|          5|AL111995|     jerry|           1995|     tropical storm|              FL|
|          6|AL051960|     donna|           1960|          hurricane|              NC|
|          7|AL011995|   allison|           1995|     tropical storm|              FL|
|          8|AL051958|      ella|          

In [65]:
# sanity check - use any or all names: 
#     bonnie (expect 5 records), 
#     andrew (expect 2 records), 
#     sandy (expect 1 record), 
#     elsa (expect 3 records) 
#     katrina (expect 2 records)

test = storms_location_dim.filter((F.col("storm_name") == 'bonnie'))
test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+----------+---------------+-------------------+----------------+
|location_id|storm_id|storm_name|max(storm_year)|         storm_type|storm_state_code|
+-----------+--------+----------+---------------+-------------------+----------------+
|         29|AL032010|    bonnie|           2010|     tropical storm|              FL|
|         51|AL022004|    bonnie|           2004|     tropical storm|              FL|
|        101|AL021986|    bonnie|           1986|          hurricane|              TX|
|        182|AL021998|    bonnie|           1998|          hurricane|              NC|
|        210|AL022016|    bonnie|           2016|tropical depression|              SC|
+-----------+--------+----------+---------------+-------------------+----------------+

In [66]:
# write to s3

storms_location_dim.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/storms_location_dim/storms_location_dim.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## storm_severity_dim

In [67]:
# select named storms with catergories

storms_severity_dim = namedStorms \
    .select(['storm_id', 
             'storm_name',
             'storm_year',
#              'storm_type',
             'category',
            'max_sustained_wind(kt)'         
            ]) \
    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
# groupby the max max_winds and drop dups

storms_severity_dim = storms_severity_dim \
    .groupBy(['storm_id',
             'storm_name',
             'storm_year',
             'category' 
             ]).agg(F.max("max_sustained_wind(kt)")) \
    .dropDuplicates()

storms_severity_dim.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+----------+----------------------+---------------------------+
|storm_id|storm_name|storm_year|category              |max(max_sustained_wind(kt))|
+--------+----------+----------+----------------------+---------------------------+
|AL111959|judith    |1959      |1                     |75.0                       |
|AL101951|item      |1951      |tropical storm        |55.0                       |
|AL081975|doris     |1975      |tropical storm        |60.0                       |
|AL081950|how       |1950      |tropical storm        |40.0                       |
|AL102018|joyce     |2018      |tropical storm        |45.0                       |
|AL012010|alex      |2010      |1                     |80.0                       |
|AL052008|edouard   |2008      |tropical depression   |30.0                       |
|AL031996|cesar     |1996      |1                     |75.0                       |
|AL092016|hermine   |2016      |extratropical storm   |60.0                 

In [69]:
# create an 'index'

storms_severity_dim = storms_severity_dim \
    .withColumn("severity_id", \
                F.monotonically_increasing_id())

win = W.orderBy("severity_id")

storms_severity_dim = storms_severity_dim \
    .withColumn("severity_id", \
                F.row_number().over(win)-1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# reorder and  show

storms_severity_dim = storms_severity_dim \
    .select(['severity_id', 
             'storm_id',
             'storm_name',
             'storm_year',
             'category',
             col('max(max_sustained_wind(kt))').alias('max_sustained_wind(kt)') \
            ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# sanity check - use any or all names: 
#     andrew (expect 9 records)
#     fred (expect 14 records), 
#     elsa (expect 4 records) 
#     florence (expect 40 records), 
#     sandy (expect 7 record), 

test = storms_severity_dim.filter((F.col("storm_name") == 'sandy'))
test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+----------+----------+--------------------+----------------------+
|severity_id|storm_id|storm_name|storm_year|            category|max_sustained_wind(kt)|
+-----------+--------+----------+----------+--------------------+----------------------+
|        300|AL182012|     sandy|      2012| tropical depression|                  30.0|
|        853|AL182012|     sandy|      2012|                   2|                  95.0|
|        902|AL182012|     sandy|      2012|                   1|                  80.0|
|       1014|AL182012|     sandy|      2012|low - not tropica...|                  25.0|
|       1046|AL182012|     sandy|      2012| extratropical storm|                  75.0|
|       2275|AL182012|     sandy|      2012|      tropical storm|                  60.0|
|       2408|AL182012|     sandy|      2012|                   3|                 100.0|
+-----------+--------+----------+----------+--------------------+----------------------+

In [72]:
# write to s3
 
storms_severity_dim.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/storms_severity_dim/storms_severity_dim.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## storms_metadata_fact

In [73]:
# join dfs

name = storms_byName_dim \
    .select(['storm_id',
             'storm_name',
             'storm_year'
            ])

sev = storms_severity_dim \
    .select([ \
             'storm_id',
             'storm_name',
             F.col('storm_year').alias('sev_storm_year'),
             'severity_id',
             'category'
            ])


loc = storms_location_dim \
    .select([ \
             'storm_id',
             'storm_name',
             col('max(storm_year)').alias('loc_storm_year'),
             'location_id',
             'storm_state_code'
            ])


storms_metadata_fact = name \
            .join(sev, ['storm_id', 'storm_name'], 'inner') \
            .join(loc, ['storm_id', 'storm_name'], 'inner') \
            .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
storms_metadata_fact.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+----------+--------------+-----------+--------------------+--------------+-----------+----------------+
|storm_id|storm_name|storm_year|sev_storm_year|severity_id|            category|loc_storm_year|location_id|storm_state_code|
+--------+----------+----------+--------------+-----------+--------------------+--------------+-----------+----------------+
|AL011966|      alma|      1966|          1966|          3|      tropical storm|          1966|        117|              FL|
|AL092016|   hermine|      2016|          2016|          4| extratropical storm|          2016|        126|              FL|
|AL122005|   katrina|      2005|          2005|         10| extratropical storm|          2005|        135|              LA|
|AL122005|   katrina|      2005|          2005|         10| extratropical storm|          2005|        102|              FL|
|AL111959|    judith|      1959|          1959|         11|                   1|          1959|         87|              FL|


In [75]:
# groupby category

storms_metadata_fact = storms_metadata_fact \
    .groupBy(['storm_id',
             'storm_name',
             'storm_year',
             'severity_id',
             'location_id',
             'storm_state_code' 
             ]).agg(F.max("category"))\
    .dropDuplicates()
    
storms_metadata_fact.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+----------+-----------+-----------+----------------+-----------------------------------------------+
|storm_id|storm_name|storm_year|severity_id|location_id|storm_state_code|max(category)                                  |
+--------+----------+----------+-----------+-----------+----------------+-----------------------------------------------+
|AL011953|alice     |1953      |362        |200        |FL              |tropical depression                            |
|AL011953|alice     |1953      |1022       |200        |FL              |tropical storm                                 |
|AL011955|brenda    |1955      |1788       |213        |LA              |tropical depression                            |
|AL011955|brenda    |1955      |2865       |213        |LA              |tropical storm                                 |
|AL011959|arlene    |1959      |1846       |48         |LA              |tropical storm                                 |
|AL011959|arlene    |195

In [76]:
# create an 'index'

storms_metadata_fact = storms_metadata_fact \
    .withColumn("storm_meta_id", \
                F.monotonically_increasing_id())

win = W.orderBy("storm_meta_id")

storms_metadata_fact = storms_metadata_fact \
    .withColumn("storm_meta_id", \
                F.row_number().over(win)-1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
# reorder
storms_metadata_fact = storms_metadata_fact \
    .select(['storm_meta_id', 
             'storm_id',
             'storm_name',
             'storm_year',
             'severity_id',
             col('max(category)').alias('category'),
             'location_id',
             'storm_state_code'
            ])

storms_metadata_fact.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+----------+----------+-----------+--------------------+-----------+----------------+
|storm_meta_id|storm_id|storm_name|storm_year|severity_id|            category|location_id|storm_state_code|
+-------------+--------+----------+----------+-----------+--------------------+-----------+----------------+
|            0|AL011953|     alice|      1953|        373| tropical depression|        200|              FL|
|            1|AL011953|     alice|      1953|       1015|      tropical storm|        200|              FL|
|            2|AL011955|    brenda|      1955|       1789| tropical depression|        213|              LA|
|            3|AL011955|    brenda|      1955|       2871|      tropical storm|        213|              LA|
|            4|AL011959|    arlene|      1959|       1843|      tropical storm|         48|              LA|
|            5|AL011959|    arlene|      1959|       2766| tropical depression|         48|              LA|
|            6|AL01

In [78]:
# sanity check - use any or all names: 
#     fred (expect 5 records)
#     carol (expect 12 records), 
#     elsa (expect 12 records) 

test = storms_metadata_fact.filter(F.col("storm_name") == 'elsa')
test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+----------+----------+-----------+-------------------+-----------+----------------+
|storm_meta_id|storm_id|storm_name|storm_year|severity_id|           category|location_id|storm_state_code|
+-------------+--------+----------+----------+-----------+-------------------+-----------+----------------+
|          449|AL052021|      elsa|      2021|        399|     tropical storm|        183|              FL|
|          450|AL052021|      elsa|      2021|        399|     tropical storm|        193|              NY|
|          451|AL052021|      elsa|      2021|        399|     tropical storm|        219|              RI|
|          452|AL052021|      elsa|      2021|       1353|extratropical storm|        183|              FL|
|          453|AL052021|      elsa|      2021|       1353|extratropical storm|        193|              NY|
|          454|AL052021|      elsa|      2021|       1353|extratropical storm|        219|              RI|
|          455|AL052021|    

In [79]:
# write to s3

storms_metadata_fact.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/storms_metadata_fact/storms_metadata_fact.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## storm_babyNames_fact

In [80]:
# join storms metadata fact table to baby names fact
meta = storms_metadata_fact \
    .select(['storm_meta_id',
            'storm_id',
            'storm_name',
            'storm_year',
            'storm_state_code',
            'category',
            'storm_meta_id',
            'storm_id',
            'location_id',
            'severity_id'
            ])

bby = babyNames_byState_fact \
      .select(['babyName_id', 
              'birth_name',
              'birth_year',
              'birth_state_code',
              'popularity',
              'gender'
            ])


on = [(meta.storm_name == bby.birth_name)]
storms_babyNames_fact = meta \
            .join(bby, on, 'inner') \
            .select(meta['storm_meta_id']
                    ,meta['storm_id'] \
                    ,bby['babyName_id']\
                    ,meta['storm_id'] \
                    ,meta['storm_name'] \
                    ,bby['birth_name'] \
                    ,bby['gender'] \
                    ,bby['birth_year'] \
                    ,bby['popularity'] \
                    ,meta['storm_year'] \
                    ,meta['category'] \
                    ,meta['storm_state_code'] \
                    ,bby['birth_state_code'] \
                   ) \
            .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# create an 'index'

storms_babyNames_fact = storms_babyNames_fact \
    .withColumn("storm_babyName_id", \
                F.monotonically_increasing_id())

win = W.orderBy("storm_babyName_id")

storms_babyNames_fact = storms_babyNames_fact \
    .withColumn("storm_babyName_id", \
                F.row_number().over(win)-1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
storms_babyNames_fact = storms_babyNames_fact \
    .select(['storm_babyName_id',
            'babyName_id',
            'storm_meta_id',
            'storm_id',
            'storm_name',
            'storm_year',
            'category',
            'storm_state_code',
            'birth_year',
            'birth_name',
            'popularity',
            'birth_state_code',
            'gender'
            ]).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
storms_babyNames_fact.show(5)
# storms_babyNames_fact.toPandas().sample(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------+-------------+--------+----------+----------+--------------+----------------+----------+----------+----------+----------------+------+
|storm_babyName_id|babyName_id|storm_meta_id|storm_id|storm_name|storm_year|      category|storm_state_code|birth_year|birth_name|popularity|birth_state_code|gender|
+-----------------+-----------+-------------+--------+----------+----------+--------------+----------------+----------+----------+----------+----------------+------+
|                0|     440544|           12|AL011968|      abby|      1968|tropical storm|              FL|      1924|      abby|         5|              NY|     F|
|                1|     478946|           12|AL011968|      abby|      1968|tropical storm|              FL|      1925|      abby|        14|              NY|     F|
|                2|     516545|           12|AL011968|      abby|      1968|tropical storm|              FL|      1926|      abby|        10|              NY|     F|
|   

In [84]:
# write to s3

storms_babyNames_fact.write.mode('overwrite').csv('s3://capstonesources/OUTPUT/storms_babyNames_fact/storms_babyNames_fact.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Validation

In [ ]:
# check files are in bucket under partition

import boto3
s3 = boto3.resource('s3')

## bucket
bucket = s3.Bucket('s3://capstonesources/')

## list objects within a given prefix
for obj in bucket.objects.filter(Delimiter='/', Prefix='OUTPUT/'):
    print(obj.key)

In [ ]:
# match counts from capstonePANDAS.ipynb

state_ref.count()
saffir_simpson_scale.count()
babyNames_byState_fact.count()
storms_byName_dim.count()
storms_location_dim.count()
storms_severity_dim.count()
storms_metadata_fact.count()
storms_babyNames_fact.count()

In [ ]:
# emily made landfall in 2017 as a tropical storm
# looking at birth names in FL in 2017 - 2019, the popularity for the name did go down

testset = storm_babyNames_fact.select(['storm_id', 
                                'storm_name', 
                                'storm_year', 
                                'storm_state_code',
                                'birth_year', 
                                'birth_name', 
                                'popularity', 
                                'birth_state_code', 
                                'gender'])

testset = testset.select([ \
         (
            (testset['birth_name']=='emily') & 
            (testset['storm_year']== 2017) &
            (testset['storm_state_code']== 'FL')
         ) & 
         ((
          (testset['birth_year']== 2017)   &  
          (testset['birth_state_code']== 'FL')
         ) |
         (
          (testset['birth_year']== 2018)   &  
          (testset['birth_state_code']== 'FL')
         ) |
          (
          (testset['birth_year']== 2021)   &  
          (testset['birth_state_code']== 'FL')
     ))
      ])

testsetProof = testset\
    .groupby(['storm_id',
         'storm_name', 
         'storm_year', 
         'storm_state_code',
         'birth_year', 
         'birth_name',
         'birth_state_code',
         'gender']).agg(F.max('popularity'))
                        
testsetProof